In [ ]:
%pip install qdrant-client

In [ ]:
import sys
sys.path.append(r'c:\Users\ice\projects\iris')

from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import numpy as np

from iris.config.data_pipeline_config_manager import DataPipelineConfigManager

In [ ]:
data_config = DataPipelineConfigManager()
qdrant_config = data_config.qdrant_config

In [ ]:
# Connect to Qdrant Cloud
client = QdrantClient(
    url=qdrant_config.url,
    api_key=qdrant_config.api_key
)

# Test connection
try:
    collections = client.get_collections()
    print("Successfully connected to Qdrant Cloud!")
    print("Available collections:", collections)
except Exception as e:
    print("Connection failed:", str(e))

In [ ]:
# Create a test collection
collection_name = "test_collection"
vector_size = 4  # Using small vectors for demonstration

# Create collection if it doesn't exist
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

In [ ]:
# Create some sample points
vectors = [
    np.random.rand(vector_size) for _ in range(3)
]
payloads = [
    {"description": "First point"},
    {"description": "Second point"},
    {"description": "Third point"}
]

# Upload points
points = [
    PointStruct(
        id=idx,
        vector=vector.tolist(),
        payload=payload
    )
    for idx, (vector, payload) in enumerate(zip(vectors, payloads))
]

client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
# Perform a search
search_vector = np.random.rand(vector_size)
search_results = client.search(
    collection_name=collection_name,
    query_vector=search_vector,
    limit=2
)

print("Search results:")
for result in search_results:
    print(f"Point {result.id}: score={result.score}, payload={result.payload}")

In [ ]:
# Fetch a specific vector by ID
point_id = 1  # Get the second point we inserted
retrieved_point = client.retrieve(
    collection_name=collection_name,
    ids=[point_id],
    with_vectors=True  # Include the actual vector data
)

if retrieved_point:
    point = retrieved_point[0]
    print(f"Retrieved point {point_id}:")
    print(f"Vector: {point.vector}")
    print(f"Payload: {point.payload}")
else:
    print(f"No point found with ID {point_id}")

# Accessing Qdrant Cloud GUI
You can view and interact with your vectors through the Qdrant Cloud Console:
1. Go to https://cloud.qdrant.io/
2. Login with your credentials
3. Select your cluster
4. Navigate to the "Collections" tab to see your `test_collection`

In [ ]:
# Print direct URL to Qdrant dashboard
dashboard_url = f"{qdrant_config.url}/dashboard"
print(f"View your vectors in Qdrant Dashboard:\n{dashboard_url}")

# For reference, also print the full connection URL
print(f"\nCurrent connection URL:\n{qdrant_config.url}")